# ლექცია 7 - მონაცემების რამოდენიმე ფაილის გაერთიანება სხვადასხვა მეთოდით. სხვადასხვა ტრანსფორმაციები სვეტებზე. ტექსტურ მონაცემებზე მანიპულაციები.

Much of the programming work in data analysis and modeling is spent on data preparation:
loading, cleaning, transforming, and rearranging. Sometimes the way that data
is stored in files or databases is not the way you need it for a data processing application. pandas along with the Python standard library provide you with a high-level, flexible, and high-performance set of core manipulations
and algorithms to enable you to wrangle data into the right form.

### Combining and Merging Data Sets
Data contained in pandas objects can be combined together in a number of built-in
ways:

• pandas.merge connects rows in DataFrames based on one or more keys. This will
be familiar to users of SQL or other relational databases, as it implements database
join operations.

• pandas.join

• pandas.concat glues or stacks together objects along an axis.

In [5]:
import pandas as pd

In [6]:
df1 = pd.DataFrame({'ID':[1,2,3,5,9],
                    'Col_1':[1,2,3,4,5],
                    'Col_2':[6,7,8,9,10],
                    'Col_3':[11,12,13,14,15],
                    'Col_4':['apple','orange','banana','strawberry','raspberry']
                    })

df2 = pd.DataFrame({'ID':[1,1,3,5],
                    'Col_A':[8,9,10,11],
                    'Col_B':[12,13,15,17],
                    'Col_4':['apple','orange','banana','kiwi']
                   })

In [7]:
df1

,ID,Col_1,Col_2,Col_3,Col_4
0,1,1,6,11,apple
1,2,2,7,12,orange
2,3,3,8,13,banana
3,5,4,9,14,strawberry
4,9,5,10,15,raspberry


In [8]:
df2

,ID,Col_A,Col_B,Col_4
0,1,8,12,apple
1,1,9,13,orange
2,3,10,15,banana
3,5,11,17,kiwi


# pd.merge() - Database-style DataFrame Merges
* For combining data on common columns
* Most flexible, but also complex of the methods we'll discuss
* many-to-one and many-to-many joins are possible
* Side-by-side merge

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html

In [9]:
# we need to tell merge what column to merge on
# we are adding more columns to the df or placing the two columns side by side
# we only keep the rows that are common to both df's (Inner Join - this is default)
# in this example df1 is the left df and df2 is the right df (this can be changed by switching the order)
# if specify left_on and right_on with index you get key_0 this can be avoided with left_index=True, right_index=True
# one-to-many = multiple 1's in df2 merged with the 1 on df1 (df1 repeated it's 1 row to match df2's 1's)
# left_on=df1.index, right_on=df2.index    # left_index=True, right_index=True
# default is inner every column with same name - if they are different dtypes you will get an error 

In [11]:
df1

,ID,Col_1,Col_2,Col_3,Col_4
0,1,1,6,11,apple
1,2,2,7,12,orange
2,3,3,8,13,banana
3,5,4,9,14,strawberry
4,9,5,10,15,raspberry


In [12]:
df2

,ID,Col_A,Col_B,Col_4
0,1,8,12,apple
1,1,9,13,orange
2,3,10,15,banana
3,5,11,17,kiwi


In [10]:
df1.merge(df2)

,ID,Col_1,Col_2,Col_3,Col_4,Col_A,Col_B
0,1,1,6,11,apple,8,12
1,3,3,8,13,banana,10,15


In [14]:
df1.merge(df2, on='ID', suffixes=['_L', '_R'])

,ID,Col_1,Col_2,Col_3,Col_4_L,Col_A,Col_B,Col_4_R
0,1,1,6,11,apple,8,12,apple
1,1,1,6,11,apple,9,13,orange
2,3,3,8,13,banana,10,15,banana
3,5,4,9,14,strawberry,11,17,kiwi


In [16]:
df1.merge(df2, on='ID', how='left')


,ID,Col_1,Col_2,Col_3,Col_4_x,Col_A,Col_B,Col_4_y
0,1,1,6,11,apple,8.0,12.0,apple
1,1,1,6,11,apple,9.0,13.0,orange
2,2,2,7,12,orange,NaN,NaN,NaN
3,3,3,8,13,banana,10.0,15.0,banana
4,5,4,9,14,strawberry,11.0,17.0,kiwi
5,9,5,10,15,raspberry,NaN,NaN,NaN


In [17]:
df1.merge(df2, left_on='Col_2', right_on='Col_A')

,ID_x,Col_1,Col_2,Col_3,Col_4_x,ID_y,Col_A,Col_B,Col_4_y
0,3,3,8,13,banana,1,8,12,apple
1,5,4,9,14,strawberry,1,9,13,orange
2,9,5,10,15,raspberry,3,10,15,banana


In [18]:
df1.merge(df2, left_index=True, right_on="ID")

,ID,ID_x,Col_1,Col_2,Col_3,Col_4_x,ID_y,Col_A,Col_B,Col_4_y
0,1,2,2,7,12,orange,1,8,12,apple
1,1,2,2,7,12,orange,1,9,13,orange
2,3,5,4,9,14,strawberry,3,10,15,banana


In [19]:
df1

,ID,Col_1,Col_2,Col_3,Col_4
0,1,1,6,11,apple
1,2,2,7,12,orange
2,3,3,8,13,banana
3,5,4,9,14,strawberry
4,9,5,10,15,raspberry


In [20]:
df2

,ID,Col_A,Col_B,Col_4
0,1,8,12,apple
1,1,9,13,orange
2,3,10,15,banana
3,5,11,17,kiwi


#### INNER
in an inner join, you will lose rows that don’t have a match in the other DataFrame’s key column. <br>
#### OUTER
in an outer join (also known as a full outer join), all rows from both DataFrames will be present in the new DataFrame.<br>
#### LEFT
Using a left outer join will leave your new merged DataFrame with all rows from the left DataFrame, while discarding rows from the right DataFrame that don’t have a match in the key column of the left DataFrame.
<br>
#### RIGHT
The right join (or right outer join) is the mirror-image version of the left join. With this join, all rows from the right DataFrame will be retained, while rows in the left DataFrame without a match in the key column of the right DataFrame will be discarded.


# df.join()
* Anything you can do with .join you can do with .merge

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html

In [25]:
df1.join(df2, how='inner', lsuffix='_L', rsuffix='_R')

,ID_L,Col_1,Col_2,Col_3,Col_4_L,ID_R,Col_A,Col_B,Col_4_R
0,1,1,6,11,apple,1,8,12,apple
1,2,2,7,12,orange,1,9,13,orange
2,3,3,8,13,banana,3,10,15,banana
3,5,4,9,14,strawberry,5,11,17,kiwi


# pd.concat()
* good for side by side and stacking on top of each other


Concatenation is a bit different from the merging techniques you saw above. With merging, you can expect the resulting dataset to have rows from the parent datasets mixed in together, often based on some commonality. Depending on the type of merge, you might also lose rows that don’t have matches in the other dataset.

With concatenation, your datasets are just stitched together along an axis — either the row axis or column axis.
https://pandas.pydata.org/docs/reference/api/pandas.concat.html 

In [28]:
pd.concat([df1, df2], axis=1)

,ID,Col_1,Col_2,Col_3,Col_4,ID,Col_A,Col_B,Col_4
0,1,1,6,11,apple,1.0,8.0,12.0,apple
1,2,2,7,12,orange,1.0,9.0,13.0,orange
2,3,3,8,13,banana,3.0,10.0,15.0,banana
3,5,4,9,14,strawberry,5.0,11.0,17.0,kiwi
4,9,5,10,15,raspberry,NaN,NaN,NaN,NaN


# df.append()

In [29]:
df1.append(df2)

C:\Users\student\AppData\Local\Temp\ipykernel_1844\3062608662.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1.append(df2)


,ID,Col_1,Col_2,Col_3,Col_4,Col_A,Col_B
0,1,1.0,6.0,11.0,apple,NaN,NaN
1,2,2.0,7.0,12.0,orange,NaN,NaN
2,3,3.0,8.0,13.0,banana,NaN,NaN
3,5,4.0,9.0,14.0,strawberry,NaN,NaN
4,9,5.0,10.0,15.0,raspberry,NaN,NaN
0,1,NaN,NaN,NaN,apple,8.0,12.0
1,1,NaN,NaN,NaN,orange,9.0,13.0
2,3,NaN,NaN,NaN,banana,10.0,15.0
3,5,NaN,NaN,NaN,kiwi,11.0,17.0


## Class Exercise

In [ ]:
#use the functions ion the List1.csv and list2.csv files

In [30]:
df1 = pd.read_csv('List1.csv')
df2 = pd.read_csv('List2.csv')

In [31]:
df1

,FellowshipID,FirstName,Skills
0,1001,Frodo,Hiking
1,1002,Samwise,Gardening
2,1003,Gandalf,Spells
3,1004,Pippin,Cycling


In [32]:
df2

,FellowshipID,FirstName,Age
0,1001,Frodo,50
1,1002,Samwise,39
2,1006,Legolas,25
3,1007,Elrond,43
4,1008,Barromir,51


In [37]:
df3 = df1.merge(df2, on="FellowshipID", how='outer')

In [36]:
pd.concat([df1,df2])

,FellowshipID,FirstName,Skills,Age
0,1001,Frodo,Hiking,NaN
1,1002,Samwise,Gardening,NaN
2,1003,Gandalf,Spells,NaN
3,1004,Pippin,Cycling,NaN
0,1001,Frodo,NaN,50.0
1,1002,Samwise,NaN,39.0
2,1006,Legolas,NaN,25.0
3,1007,Elrond,NaN,43.0
4,1008,Barromir,NaN,51.0


## Rename Columns

In [38]:
df3

,FellowshipID,FirstName_x,Skills,FirstName_y,Age
0,1001,Frodo,Hiking,Frodo,50.0
1,1002,Samwise,Gardening,Samwise,39.0
2,1003,Gandalf,Spells,NaN,NaN
3,1004,Pippin,Cycling,NaN,NaN
4,1006,NaN,NaN,Legolas,25.0
5,1007,NaN,NaN,Elrond,43.0
6,1008,NaN,NaN,Barromir,51.0


In [40]:
df3.rename({'FirstName_x': 'Name'}, axis=1)

,FellowshipID,Name,Skills,FirstName_y,Age
0,1001,Frodo,Hiking,Frodo,50.0
1,1002,Samwise,Gardening,Samwise,39.0
2,1003,Gandalf,Spells,NaN,NaN
3,1004,Pippin,Cycling,NaN,NaN
4,1006,NaN,NaN,Legolas,25.0
5,1007,NaN,NaN,Elrond,43.0
6,1008,NaN,NaN,Barromir,51.0


## Reverse column order

In [42]:
df3.loc[:,::-1]

,Age,FirstName_y,Skills,FirstName_x,FellowshipID
0,50.0,Frodo,Hiking,Frodo,1001
1,39.0,Samwise,Gardening,Samwise,1002
2,NaN,NaN,Spells,Gandalf,1003
3,NaN,NaN,Cycling,Pippin,1004
4,25.0,Legolas,NaN,NaN,1006
5,43.0,Elrond,NaN,NaN,1007
6,51.0,Barromir,NaN,NaN,1008


### Select columns by data type

In [43]:
df3.dtypes

FellowshipID      int64
FirstName_x      object
Skills           object
FirstName_y      object
Age             float64
dtype: object

In [46]:
df3.select_dtypes(include=['int64', 'float64'])

,FellowshipID,Age
0,1001,50.0
1,1002,39.0
2,1003,NaN
3,1004,NaN
4,1006,25.0
5,1007,43.0
6,1008,51.0


In [47]:
df3.select_dtypes(exclude=['int64'])

,FirstName_x,Skills,FirstName_y,Age
0,Frodo,Hiking,Frodo,50.0
1,Samwise,Gardening,Samwise,39.0
2,Gandalf,Spells,NaN,NaN
3,Pippin,Cycling,NaN,NaN
4,NaN,NaN,Legolas,25.0
5,NaN,NaN,Elrond,43.0
6,NaN,NaN,Barromir,51.0


## Turn numbers into categories 

In [50]:
df = pd.read_csv('Salaries.csv')
df.head(20)


,rank,discipline,phd,service,sex,salary
0,Prof,B,56,49,Male,186960
1,Prof,A,12,6,Male,93000
2,Prof,A,23,20,Male,110515
3,Prof,A,40,31,Male,131205
4,Prof,B,20,18,Male,104800
5,Prof,A,20,20,Male,122400
6,AssocProf,A,20,17,Male,81285
7,Prof,A,18,18,Male,126300
8,Prof,A,29,19,Male,94350
9,Prof,A,51,51,Male,57800


In [56]:
df.size

468

In [58]:
min_value = df['salary'].min()

In [59]:
max_value = df['salary'].max()

In [61]:
df['categories'] = pd.cut(df['salary'], bins=[min_value, 100000, 150000, max_value], labels=['Low', 'Medium', 'High'])

In [62]:
df

,rank,discipline,phd,service,sex,salary,categories
0,Prof,B,56,49,Male,186960,High
1,Prof,A,12,6,Male,93000,Low
2,Prof,A,23,20,Male,110515,Medium
3,Prof,A,40,31,Male,131205,Medium
4,Prof,B,20,18,Male,104800,Medium
...,...,...,...,...,...,...,...
73,Prof,B,18,10,Female,105450,Medium
74,AssocProf,B,19,6,Female,104542,Medium
75,Prof,B,17,17,Female,124312,Medium
76,Prof,A,28,14,Female,109954,Medium
